In [1358]:
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np
import random
import re

Load the dataset into a Pandas DataFrame.

In [1359]:
#Load the data
file1 = 'C:/Users/.../Video-Game-Data-Analysis/Data/3 Juegos en steam.csv'
file2 = 'C:/Users/.../Video-Game-Data-Analysis/Data/4. Video Games Sales.csv'
file3 = 'C:/Users/.../Video-Game-Data-Analysis/Data/2 Console_sales.csv'
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)
df3 = pd.read_excel(file3)

Visualize the data

In [1360]:
# show data
df1.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


Explore the initial records and descriptive statistics of the dataset in df1

In [1361]:
#Preview tipe of data 
def summary(df1):
    print(f'data shape: {df1.shape}')
    summ = pd.DataFrame(df1.dtypes, columns=['data type'])
    summ['#missing'] = df1.isnull().sum()
    summ['%missing'] = df1.isnull().sum()*100/len(df1)
    summ['#unique'] = df1.nunique()
    desc = pd.DataFrame(df1.describe(include='all').T)
    summ['min'] = desc['min']
    summ['max'] = desc['max']
    return summ

summary(df1)

data shape: (27075, 18)


,data type,#missing,%missing,#unique,min,max
appid,int64,0,0.000000,27075,10.0,1069460.0
name,object,0,0.000000,27033,NaN,NaN
release_date,object,0,0.000000,2619,NaN,NaN
english,int64,0,0.000000,2,0.0,1.0
developer,object,1,0.003693,17112,NaN,NaN
publisher,object,14,0.051708,14353,NaN,NaN
platforms,object,0,0.000000,7,NaN,NaN
required_age,int64,0,0.000000,6,0.0,18.0
categories,object,0,0.000000,3333,NaN,NaN
genres,object,0,0.000000,1552,NaN,NaN


In [1362]:
#View data from the genres column
unique_genres = df1['genres'].unique()
print(unique_genres)

['Action' 'Action;Free to Play' 'Action;Free to Play;Strategy' ...
 'Action;Adventure;Indie;Massively Multiplayer;RPG;Strategy;Early Access'
 'Action;Adventure;Casual;Free to Play;Indie;RPG;Simulation;Sports;Strategy'
 'Casual;Free to Play;Massively Multiplayer;RPG;Early Access']


Perform feature engineering to create new relevant variables.

In [1363]:
# Transform data of column genres
def transform_genres(row):
    genre_list = row['genres'].split(';')  # Split genres into a list

    if 'MOBA' in row['steamspy_tags']:
        return 'MOBA'

    if len(genre_list) == 1:
        return genre_list[0]
    elif 'Action' in genre_list and 'RPG' in genre_list and 'Strategy' in genre_list:
        return 'Tactical-ARPG'
    elif 'Action' in genre_list and 'RPG' in genre_list:
        return 'ARPG'
    elif 'Action' in genre_list and 'Adventure' in genre_list:
        return 'Action-Adventure'
    
    # Add the additional condition to choose the first tag other than those mentioned..
    for item in genre_list:
        if item not in ['Violent', 'Nudity', 'Free to Play', 'Massively Multiplayer', 'Sexual Content', 'Early Access']:
            return item

    # If none of the above conditions are met, return the first element of the list.
    return genre_list[0]

# Apply the function to the rows of the DataFrame.
df1['genres'] = df1.apply(transform_genres, axis=1)

In [1364]:
unique_genres = df1['genres'].unique()
print(unique_genres)

['Action' 'MOBA' 'Action-Adventure' 'Indie' 'Strategy' 'RPG'
 'Animation & Modeling' 'ARPG' 'Casual' 'Simulation' 'Racing' 'Adventure'
 'Tactical-ARPG' 'Sports' 'Utilities' 'Free to Play'
 'Design & Illustration' 'Gore' 'Education' 'Web Publishing'
 'Audio Production' 'Photo Editing' 'Early Access' 'Accounting'
 'Video Production' 'Sexual Content' 'Violent' 'Software Training'
 'Massively Multiplayer']


In [1365]:
# Change the format of the 'release_date' column to just the year.
df1['release_date'] = pd.to_datetime(df1['release_date']).dt.year

# Change all values ​​in the 'platforms' column to 'PC'.
df1['platforms'] = 'PC-STEAM'

# Map the 'required_age' values ​​to the corresponding categories.
age_mapping = {
    0: 'E',
    3: 'E',
    7: 'E10+',
    12: 'T',
    16: 'M',
    18: 'AO'
}
df1['required_age'] = df1['required_age'].map(age_mapping)

# Define a function to generate random values ​​based on the specified range.
def random_owners(owners_range):
    min_val, max_val = map(int, owners_range.split('-'))
    return random.randint(min_val, max_val)


# Apply the function to the 'owners' column to convert the values ​​and assign random values.
df1['owners'] = df1['owners'].apply(random_owners)

# Create new column 'Global_Sales' containing multiplication of 'owners' and 'price.
df1['Global_Sales'] = (df1['owners'] * df1['price']) / 1000000
df1['Global_Sales'] = df1['Global_Sales'].round(2)

# Calculate rating average.
df1['average_rating'] = (df1['positive_ratings'] / (df1['positive_ratings'] + df1['negative_ratings'])*10).fillna(0).round(1)



In [1366]:
#Drop columns
columns_to_drop = ['appid', 'english', 'categories', 'achievements', 'publisher', 'steamspy_tags', 'positive_ratings', 'negative_ratings', 'median_playtime', 'owners', 'price']
df1 = df1.drop(columns_to_drop, axis=1)

In [1367]:
new_names = {'name': 'Name', 'release_date': 'Release', 'developer': 'Developer', 'platforms': 'Platform', 'required_age': 'Rating', 'genres': 'Genres', 'average_playtime': 'Average_Playtime', 'average_rating': 'Score'}

# rename columns
df1.rename(columns=new_names, inplace=True)

Visualize the data changes

In [1368]:
df1.head()

,Name,Release,Developer,Platform,Rating,Genres,Average_Playtime,Global_Sales,Score
0,Counter-Strike,2000,Valve,PC-STEAM,E,Action,17612,98.20,9.7
1,Team Fortress Classic,1999,Valve,PC-STEAM,E,Action,277,39.27,8.4
2,Day of Defeat,2003,Valve,PC-STEAM,E,Action,187,39.54,9.0
3,Deathmatch Classic,2001,Valve,PC-STEAM,E,Action,258,34.95,8.3
4,Half-Life: Opposing Force,1999,Gearbox Software,PC-STEAM,E,Action,624,31.55,9.5


Export to a csv file

In [1369]:
archivo_csv = 'steam_data.csv'

# Export to csv file.
df1.to_csv(archivo_csv, index=False)

Visualize the data file2

In [1370]:
df2.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,NaN,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,NaN,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


Explore the initial records and descriptive statistics of the dataset in df2

In [1371]:
def summary(df2):
    print(f'data shape: {df2.shape}')
    summ = pd.DataFrame(df2.dtypes, columns=['data type'])
    summ['#missing'] = df2.isnull().sum()
    summ['%missing'] = df2.isnull().sum()*100/len(df2)
    summ['#unique'] = df2.nunique()
    desc = pd.DataFrame(df2.describe(include='all').T)
    summ['min'] = desc['min']
    summ['max'] = desc['max']
    return summ

summary(df2)

data shape: (16719, 16)


,data type,#missing,%missing,#unique,min,max
Name,object,2,0.011962,11562,NaN,NaN
Platform,object,0,0.000000,31,NaN,NaN
Year_of_Release,float64,269,1.608948,39,1980.0,2020.0
Genre,object,2,0.011962,12,NaN,NaN
Publisher,object,54,0.322986,581,NaN,NaN
NA_Sales,float64,871,5.209642,392,0.0,41.36
EU_Sales,float64,1199,7.171482,302,0.0,28.96
JP_Sales,float64,0,0.000000,244,0.0,10.22
Other_Sales,float64,1378,8.242120,148,0.0,10.57
Global_Sales,float64,991,5.927388,613,0.01,82.53


Perform feature engineering to create new relevant variables.

In [1372]:
# Replace the "Role-Playing" values ​​with "RPG" in the "Genre" column.
df2['Genre'] = df2['Genre'].replace('Role-Playing', 'RPG')
# Change the format of the 'Year_of_Release' column to just the year and type float.
df2['Year_of_Release'] = df2['Year_of_Release'].apply(lambda x: int(x) if not pd.isna(x) else pd.NA)


In [1373]:
# Map the values ​​of the 'Rating' column to the corresponding categories.
# Clear the 'Rating' column of unwanted characters
df2['Rating'] = df2['Rating'].str.strip()  
df2['Rating'] = df2['Rating'].replace('', pd.NA)  

# Create the value mapping and consider that the NaNs represent 'RP'
rating_mapping = {
    'EC': 'E',
    'K-A': 'E',
    'E': 'E',
    'E10+': 'E10+',
    'M': 'M',
    'AO': 'AO',
    'T': 'T',
    pd.NA: 'RP',  
    'RP': 'RP'
}

df2['Rating'] = df2['Rating'].map(rating_mapping)

# Define conditions and new values
conditions = [
    (df2['Rating'].isna() | (df2['Rating'] == '')) & (df2['Platform'] == '2600'),
    (df2['Rating'].isna() | (df2['Rating'] == '')) & (df2['Platform'] == 'NES'),
    (df2['Rating'].isna() | (df2['Rating'] == '')) & (df2['Platform'] == 'GB'),
    (df2['Rating'].isna() | (df2['Rating'] == '')) & (df2['Platform'] == 'GBA'),
    (df2['Rating'].isna() | (df2['Rating'] == '')) & (df2['Platform'] == 'GC'),
    (df2['Rating'].isna() | (df2['Rating'] == '')) & (df2['Platform'] == 'GEN'),
    (df2['Rating'].isna() | (df2['Rating'] == '')) & (df2['Platform'] == 'NG'),
    (df2['Rating'].isna() | (df2['Rating'] == '')) & ((df2['Platform'] == 'SNES') & (df2['Genre'] != 'Shooter')),
    (df2['Rating'].isna() | (df2['Rating'] == '')) & ((df2['Platform'] == 'SNES') & (df2['Genre'] != 'Fighting')),
    (df2['Rating'].isna() | (df2['Rating'] == '')) & ((df2['Platform'] == 'SNES') & (df2['Genre'] == 'Shooter')),
    (df2['Rating'].isna() | (df2['Rating'] == '')) & ((df2['Platform'] == 'SNES') & (df2['Genre'] == 'Fighting')),
    (df2['Rating'].isna() | (df2['Rating'] == '')) & ((df2['Platform'] == 'N64') & (df2['Genre'] != 'Shooter')),
    (df2['Rating'].isna() | (df2['Rating'] == '')) & ((df2['Platform'] == 'N64') & (df2['Genre'] != 'Fighting')),
    (df2['Rating'].isna() | (df2['Rating'] == '')) & ((df2['Platform'] == 'N64') & (df2['Genre'] == 'Shooter')),
    (df2['Rating'].isna() | (df2['Rating'] == '')) & ((df2['Platform'] == 'N64') & (df2['Genre'] == 'Fighting')),
    (df2['Rating'].isna() | (df2['Rating'] == '')) & ((df2['Platform'] == 'PS') & (df2['Genre'] != 'Shooter')),
    (df2['Rating'].isna() | (df2['Rating'] == '')) & ((df2['Platform'] == 'PS') & (df2['Genre'] != 'Fighting')),
    (df2['Rating'].isna() | (df2['Rating'] == '')) & ((df2['Platform'] == 'PS') & (df2['Genre'] == 'Shooter')),
    (df2['Rating'].isna() | (df2['Rating'] == '')) & ((df2['Platform'] == 'PS') & (df2['Genre'] == 'Fighting')),
    (df2['Rating'].isna() | (df2['Rating'] == '')) & ((df2['Platform'] == 'GC') & (df2['Genre'] != 'Shooter')),
    (df2['Rating'].isna() | (df2['Rating'] == '')) & ((df2['Platform'] == 'GC') & (df2['Genre'] != 'Fighting')),
    (df2['Rating'].isna() | (df2['Rating'] == '')) & ((df2['Platform'] == 'GC') & (df2['Genre'] == 'Shooter')),
    (df2['Rating'].isna() | (df2['Rating'] == '')) & ((df2['Platform'] == 'GC') & (df2['Genre'] == 'Fighting')),
    (df2['Rating'].isna() | (df2['Rating'] == '')) & ((df2['Platform'] == 'DS') & (df2['Genre'] != 'Shooter')),
    (df2['Rating'].isna() | (df2['Rating'] == '')) & ((df2['Platform'] == 'DS') & (df2['Genre'] != 'Fighting')),
    (df2['Rating'].isna() | (df2['Rating'] == '')) & ((df2['Platform'] == 'DS') & (df2['Genre'] == 'Shooter')),
    (df2['Rating'].isna() | (df2['Rating'] == '')) & ((df2['Platform'] == 'DS') & (df2['Genre'] == 'Fighting')),
    (df2['Rating'].isna() | (df2['Rating'] == '')) & ((df2['Platform'] == '3DS') & (df2['Genre'] != 'Shooter')),
    (df2['Rating'].isna() | (df2['Rating'] == '')) & ((df2['Platform'] == '3DS') & (df2['Genre'] != 'Fighting')),
    (df2['Rating'].isna() | (df2['Rating'] == '')) & ((df2['Platform'] == '3DS') & (df2['Genre'] == 'Shooter')),
    (df2['Rating'].isna() | (df2['Rating'] == '')) & ((df2['Platform'] == '3DS') & (df2['Genre'] == 'Fighting')),
    (df2['Rating'].isna() | (df2['Rating'] == '')) & ((df2['Platform'] == 'PSP') & (df2['Genre'] == 'Shooter')),
    (df2['Rating'].isna() | (df2['Rating'] == '')) & ((df2['Platform'] == 'PSP') & (df2['Genre'] == 'Fighting')),
    (df2['Rating'].isna() | (df2['Rating'] == '')) & ((df2['Platform'] == 'PSV') & (df2['Genre'] == 'Shooter')),
    (df2['Rating'].isna() | (df2['Rating'] == '')) & ((df2['Platform'] == 'PSV') & (df2['Genre'] == 'Fighting')),
    (df2['Rating'].isna() | (df2['Rating'] == '')) & (df2['Genre'] == 'Racing'),
    (df2['Rating'].isna() | (df2['Rating'] == '')) & (df2['Genre'] == 'Strategy'),
    (df2['Rating'].isna() | (df2['Rating'] == '')) & (df2['Genre'] == 'Platform'),
    (df2['Rating'].isna() | (df2['Rating'] == '')) & ((df2['Genre'] == 'Simulation') & (df2['Publisher'] == 'Konami Digital Entertainment'))
]

new_ratings = [
    'E', 'E', 'E', 'E', 'E', 'E', 'T', 'E', 'E', 'T', 'T', 'E', 'E', 'T', 'T', 'E', 'E', 'T', 'T', 'E10+', 'E10+', 'T', 'T', 'E10+', 'E10+', 'T', 'T', 'E10+', 'E10+', 'T', 'T', 'E10+', 'T', 'E', 'T', 'T', 'T', 'T', 'T'
]

# Use np.select to update the 'Rating' column based on conditions
df2['Rating'] = np.select(conditions, new_ratings, df2['Rating'])


In [1374]:
uniquevalues = df2['Platform'].unique()
print(uniquevalues)

['Wii' 'NES' 'GB' 'DS' 'X360' 'PS3' 'PS2' 'SNES' 'GBA' 'PS4' '3DS' 'N64'
 'PS' 'XB' 'PC' '2600' 'PSP' 'XOne' 'WiiU' 'GC' 'GEN' 'DC' 'PSV' 'SAT'
 'SCD' 'WS' 'NG' 'TG16' '3DO' 'GG' 'PCFX']


In [1375]:
platform_mapping = {
    'Wii': 'Nintendo Wii',
    'GEN': 'Sega Genesys',
    'DC': 'Sega Dreamcast',
    'WiiU': 'Nintendo Wii U',
    '3DS': 'Nintendo 3DS',
    'DS': 'Nintendo DS',
    'PS': 'Sony PS',
    'PS2': 'Sony PS2',
    'PS3': 'Sony PS3',
    'PS4': 'Sony PS4',
    'PSV': 'Sony PS Vita',
    'PSP': 'Sony PSP',
    'X360': 'Xbox360',
    'XOne': 'Xbox One'
}

df2['Platform'] = df2['Platform'].map(platform_mapping).fillna(df2['Platform'])

In [1376]:
uniquevalues = df2['Platform'].unique()
print(uniquevalues)

['Nintendo Wii' 'NES' 'GB' 'Nintendo DS' 'Xbox360' 'Sony PS3' 'Sony PS2'
 'SNES' 'GBA' 'Sony PS4' 'Nintendo 3DS' 'N64' 'Sony PS' 'XB' 'PC' '2600'
 'Sony PSP' 'Xbox One' 'Nintendo Wii U' 'GC' 'Sega Genesys'
 'Sega Dreamcast' 'Sony PS Vita' 'SAT' 'SCD' 'WS' 'NG' 'TG16' '3DO' 'GG'
 'PCFX']


In [1377]:
# Defines a dictionary to maps Publishers to Developers
publisher_to_developer = {
    'Capcom': 'Capcom',
    'Disney Interactive Studios': 'Disney Interactive Studios',
    'DreamWorks Interactive': 'Dreamworks',
    'Electronic Arts': 'EA',
    'Electronic Arts Victor': 'EA',
    'Konami Digital Entertainment': 'Konami',
    'LEGO Media': 'TT Games',
    'Mattel Interactive': 'Mattel Games',
    'Microsoft Game Studios': 'Microsoft',
    'Namco Bandai Games': 'Namco',
    'Nintendo': 'Nintendo',
    'Oxigen Interactive': 'Oxigen',
    'Sega': 'Sega',
    'Sony Computer Entertainment': 'Sony Computer Entertainment',
    'SquareSoft': 'Square',
    'Square Enix' : 'Square',
    'Square' : 'Square',
    'Sunsoft': 'Sunsoft',
    'Ubisoft': 'Ubisoft',
    'Ubisoft Annecy': 'Ubisoft'
    
}

# Define a function to update the Developer
def update_developer(row):
    if pd.isna(row['Developer']) or row['Developer'] == '':
        publisher = row['Publisher']
        if publisher in publisher_to_developer:
            return publisher_to_developer[publisher]
    return row['Developer']

# Apply the function to your DataFrame
df2['Developer'] = df2.apply(update_developer, axis=1)

df2['Developer'].fillna('Unknown', inplace=True)
df2['Publisher'].fillna('Unknown', inplace=True)

sales = ['NA_Sales','EU_Sales','JP_Sales','Other_Sales','Global_Sales']

sales_not_NA = ['EU_Sales','JP_Sales','Other_Sales']

sales_not_EU = ['NA_Sales','JP_Sales','Other_Sales']

sales_not_other = ['NA_Sales','EU_Sales','JP_Sales']

sales_not_global = ['NA_Sales','EU_Sales','JP_Sales','Other_Sales']

mask = df2['NA_Sales'].isna() & df2[['EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']].notna().all(axis=1)

df2.loc[mask, 'NA_Sales'] = df2[mask]['Global_Sales'] - df2[mask][sales_not_NA].sum(axis = 1)
mask = df2['EU_Sales'].isna() & df2[['NA_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']].notna().all(axis=1)

df2.loc[mask, 'EU_Sales']= df2[mask]['Global_Sales'] - df2[mask][sales_not_EU].sum(axis = 1)
mask = df2['Other_Sales'].isna() & df2[['NA_Sales','EU_Sales','JP_Sales','Global_Sales']].notna().all(axis=1)

df2.loc[mask, 'Other_Sales']= df2[mask]['Global_Sales'] - df2[mask][sales_not_other].sum(axis = 1)
mask = df2['Global_Sales'].isna() & df2[['NA_Sales','EU_Sales','JP_Sales','Other_Sales']].isna().any(axis=1)

df2[mask]

# Fill empty values ​​in the 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales' columns with 0.
df2['NA_Sales'].fillna(0, inplace=True)
df2['EU_Sales'].fillna(0, inplace=True)
df2['JP_Sales'].fillna(0, inplace=True)
df2['Other_Sales'].fillna(0, inplace=True)
df2['Global_Sales'].fillna(0, inplace=True)


df2['Global_Sales'] = df2[['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']].sum(axis=1)

In [1378]:
uniquevalues = df2['Rating'].unique()
print(uniquevalues)

['E' 'E10+' 'M' nan 'T' 'AO' 'RP']


In [1379]:
df2.dropna(subset=['Name'], inplace=True)
columns_to_drop = ['Publisher', 'Critic_Score', 'Critic_Count', 'User_Count']
df2 = df2.drop(columns_to_drop, axis=1)

In [1380]:
nuevos_nombres = {'Year_of_Release': 'Release', 'Genre': 'Genres', 'average_playtime': 'Average_Playtime', 'User_Score': 'Score'}

# Use the rename() method to change column names
df2.rename(columns=nuevos_nombres, inplace=True)

In [1381]:
column_order = ['Name', 'Platform', 'Genres', 'Release', 'Developer', 'Rating', 'Score', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']
df2 = df2[column_order]

In [1382]:
df2.head()

,Name,Platform,Genres,Release,Developer,Rating,Score,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,Wii Sports,Nintendo Wii,Sports,2006,Nintendo,E,8,41.36,28.96,3.77,8.44,82.53
1,Super Mario Bros.,NES,Platform,1985,Nintendo,E,NaN,29.08,3.58,6.81,0.77,40.24
2,Mario Kart Wii,Nintendo Wii,Racing,2008,Nintendo,E,8.3,15.68,12.76,3.79,3.29,35.52
3,Wii Sports Resort,Nintendo Wii,Sports,2009,Nintendo,E,8,15.61,10.93,3.28,2.95,32.77
4,Pokemon Red/Pokemon Blue,GB,RPG,1996,Nintendo,E,NaN,11.27,8.89,10.22,1.00,31.38


Export to a CSV file

In [1384]:
archivo_csv = 'games_console.csv'

# Exporta el DataFrame a un archivo CSV.
df2.to_csv(archivo_csv, index=False)

In [1385]:
def summary(df2):
    print(f'data shape: {df2.shape}')
    summ = pd.DataFrame(df2.dtypes, columns=['data type'])
    summ['#missing'] = df2.isnull().sum()
    summ['%missing'] = df2.isnull().sum()*100/len(df2)
    summ['#unique'] = df2.nunique()
    desc = pd.DataFrame(df2.describe(include='all').T)
    summ['min'] = desc['min']
    summ['max'] = desc['max']
    return summ

summary(df2)

data shape: (16717, 12)


,data type,#missing,%missing,#unique,min,max
Name,object,0,0.000000,11562,NaN,NaN
Platform,object,0,0.000000,31,NaN,NaN
Genres,object,0,0.000000,12,NaN,NaN
Release,object,269,1.609140,39,NaN,NaN
Developer,object,0,0.000000,1704,NaN,NaN
Rating,object,2832,16.940839,6,NaN,NaN
Score,object,6702,40.090925,96,NaN,NaN
NA_Sales,float64,0,0.000000,526,-0.01,41.36
EU_Sales,float64,0,0.000000,471,-0.01,28.96
JP_Sales,float64,0,0.000000,244,0.0,10.22


Create the new DataFrame 'games' by combining the columns of df1 and df2.

In [1386]:
games = pd.DataFrame({
    'Id_Game': range(1, len(df2) + len(df1) + 1),
    'Name': pd.concat([df2['Name'], df1['Name']], ignore_index=True),
    'Developer': pd.concat([df2['Developer'], df1['Developer']], ignore_index=True),
    'Platform': pd.concat([df2['Platform'], df1['Platform']], ignore_index=True),
    'Release': pd.concat([df2['Release'], df1['Release']], ignore_index=True),
    'Genre': pd.concat([df2['Genres'], df1['Genres']], ignore_index=True),
    'Rating': pd.concat([df2['Rating'], df1['Rating']],  ignore_index=True),
    'Score': pd.concat([df2['Score'], df1['Score']], ignore_index=True),
    'Global_Sales': pd.concat([df2['Global_Sales'], df1['Global_Sales']], ignore_index=True)
})

In [1387]:
unique_names = games['Name'].unique()
id_mapping = {name: id for id, name in enumerate(unique_names, start=1)}
games['Id_Game'] = games['Name'].map(id_mapping)

In [1388]:
# Create auxiliary table for Average_Playtime
df1_aux = df1[['Name', 'Platform', 'Average_Playtime']]

# Combine the main table "Games" with the auxiliary table "Average_Playtime" using Name and Platform
games = pd.merge(games, df1_aux, on=['Name', 'Platform'], how='left')



In [1389]:
df2_aux = df2[['Name', 'Platform', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']]

games = pd.merge(games, df2_aux, on=['Name', 'Platform'], how='left')


Visualize new dataframe

In [1390]:
games.head()

,Id_Game,Name,Developer,Platform,Release,Genre,Rating,Score,Global_Sales,Average_Playtime,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,1,Wii Sports,Nintendo,Nintendo Wii,2006,Sports,E,8,82.53,NaN,41.36,28.96,3.77,8.44
1,2,Super Mario Bros.,Nintendo,NES,1985,Platform,E,NaN,40.24,NaN,29.08,3.58,6.81,0.77
2,3,Mario Kart Wii,Nintendo,Nintendo Wii,2008,Racing,E,8.3,35.52,NaN,15.68,12.76,3.79,3.29
3,4,Wii Sports Resort,Nintendo,Nintendo Wii,2009,Sports,E,8,32.77,NaN,15.61,10.93,3.28,2.95
4,5,Pokemon Red/Pokemon Blue,Nintendo,GB,1996,RPG,E,NaN,31.38,NaN,11.27,8.89,10.22,1.00


In [1391]:
games['Score'] = pd.to_numeric(games['Score'], errors='coerce')

# Sort the 'games' DataFrame by the 'Global_Sales' column in descending order.
games = games.sort_values(by='Global_Sales', ascending=False)

# Create the 'Ranking' column with sequential values ​​starting from 1.
games['Ranking'] = range(1, len(games) + 1)

# Move the 'Ranking' column to the desired location (after the 'id_game' column).
column_order = ['Id_Game', 'Name', 'Developer', 'Platform', 'Release', 'Genre', 'Rating', 'Ranking', 'Average_Playtime', 'Score', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']
games = games[column_order]

# Reset the DataFrame index.
games = games.sort_values(by='Id_Game', ascending=True)

In [1392]:
games.head()

,Id_Game,Name,Developer,Platform,Release,Genre,Rating,Ranking,Average_Playtime,Score,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Nintendo,Nintendo Wii,2006,Sports,E,46,NaN,8.0,41.36,28.96,3.77,8.44,82.53
201,2,Super Mario Bros.,Nintendo,GB,1999,Platform,E,1087,NaN,NaN,3.40,1.30,0.15,0.22,5.07
1,2,Super Mario Bros.,Nintendo,NES,1985,Platform,E,143,NaN,NaN,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Nintendo,Nintendo Wii,2008,Racing,E,170,NaN,8.3,15.68,12.76,3.79,3.29,35.52
3,4,Wii Sports Resort,Nintendo,Nintendo Wii,2009,Sports,E,185,NaN,8.0,15.61,10.93,3.28,2.95,32.77


Export to a CSV file

In [1393]:
archivo_csv = 'games_data.csv'

# Exporta el DataFrame a un archivo CSV.
games.to_csv(archivo_csv, index=False)

In [1394]:
df3.head()

,Year,Dato,Console,Company,Sales
0,2011,anual,Nintendo 3DS,Nintendo,12560000.0
1,2012,anual,Nintendo 3DS,Nintendo,13480000.0
2,2013,anual,Nintendo 3DS,Nintendo,14310000.0
3,2014,anual,Nintendo 3DS,Nintendo,9740000.0
4,2015,anual,Nintendo 3DS,Nintendo,7330000.0


In [1395]:
games.head()

,Id_Game,Name,Developer,Platform,Release,Genre,Rating,Ranking,Average_Playtime,Score,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Nintendo,Nintendo Wii,2006,Sports,E,46,NaN,8.0,41.36,28.96,3.77,8.44,82.53
201,2,Super Mario Bros.,Nintendo,GB,1999,Platform,E,1087,NaN,NaN,3.40,1.30,0.15,0.22,5.07
1,2,Super Mario Bros.,Nintendo,NES,1985,Platform,E,143,NaN,NaN,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Nintendo,Nintendo Wii,2008,Racing,E,170,NaN,8.3,15.68,12.76,3.79,3.29,35.52
3,4,Wii Sports Resort,Nintendo,Nintendo Wii,2009,Sports,E,185,NaN,8.0,15.61,10.93,3.28,2.95,32.77


In [1396]:
def summary(games):
    print(f'data shape: {games.shape}')
    summ = pd.DataFrame(games.dtypes, columns=['data type'])
    summ['#missing'] = games.isnull().sum()
    summ['%missing'] = games.isnull().sum()*100/len(games)
    summ['#unique'] = games.nunique()
    desc = pd.DataFrame(games.describe(include='all').T)
    summ['min'] = desc['min']
    summ['max'] = desc['max']
    return summ

summary(games)

data shape: (43886, 15)


,data type,#missing,%missing,#unique,min,max
Id_Game,int64,0,0.000000,38100,1.0,38100.0
Name,object,0,0.000000,38100,NaN,NaN
Developer,object,1,0.002279,18505,NaN,NaN
Platform,object,0,0.000000,32,NaN,NaN
Release,object,270,0.615230,41,NaN,NaN
Genre,object,0,0.000000,34,NaN,NaN
Rating,object,2832,6.453083,6,NaN,NaN
Ranking,int64,0,0.000000,43886,1.0,43886.0
Average_Playtime,float64,16725,38.110103,1345,0.0,190625.0
Score,float64,9127,20.797065,101,0.0,10.0


Global Sales: What are the most successful genres in terms of global sales? Are there trends in the best-selling game genres?

In [1397]:
games_per_gender=games['Genre'].value_counts().head(10)

print(games_per_gender)

Genre
Action              9985
Adventure           6726
Casual              4476
Action-Adventure    3373
Indie               2868
Sports              2437
RPG                 1989
Misc                1750
Simulation          1559
Racing              1461
Name: count, dtype: int64


In [1398]:
# Global Sales by Gender
sales_per_gender = games.groupby('Genre')['Global_Sales'].sum().sort_values(ascending=False).head(10)


print(sales_per_gender)

Genre
Action              10581.55
Action-Adventure     7179.50
ARPG                 3392.19
Adventure            2874.96
RPG                  2606.45
Simulation           1956.79
Indie                1865.40
Strategy             1814.81
Sports               1383.38
Racing               1255.51
Name: Global_Sales, dtype: float64


Ranking: What are the best ranked games in terms of ranking? What features do the top-rated games have in common?

In [1399]:
# Ordenar el DataFrame por la columna "Ranking" en orden ascendente.
juegos_mejor_clasificados =  games.sort_values(by='Ranking').head(10)
juegos_mejor_clasificados = juegos_mejor_clasificados[['Ranking', 'Name', 'Platform']]
# Muestra los juegos mejor clasificados.
print(juegos_mejor_clasificados)

       Ranking                           Name  Platform
29596        1  PLAYERUNKNOWN'S BATTLEGROUNDS  PC-STEAM
29725        2          MONSTER HUNTER: WORLD  PC-STEAM
21448        3          ARK: Survival Evolved  PC-STEAM
19208        4             Grand Theft Auto V  PC-STEAM
39250        5                      HITMAN™ 2  PC-STEAM
37612        6     Sekiro™: Shadows Die Twice  PC-STEAM
19695        7      The Witcher® 3: Wild Hunt  PC-STEAM
18817        8               Cities: Skylines  PC-STEAM
18746        9                           Rust  PC-STEAM
23155       10       Rise of the Tomb Raider™  PC-STEAM


In [1400]:
#Critics Rating vs. Sales
correlacion_score_ventas = games[['Score', 'Global_Sales']].corr()
print(correlacion_score_ventas)

                 Score  Global_Sales
Score         1.000000      0.024679
Global_Sales  0.024679      1.000000


Create dimensional tables

In [1401]:

developer = games[["Developer"]].drop_duplicates().reset_index(drop=True)
developer["Id_Developer"] = developer.index + 1

platform = games[["Platform"]].drop_duplicates().reset_index(drop=True)
platform["Id_Platform"] = platform.index + 1

genre = games[["Genre"]].drop_duplicates().reset_index(drop=True)
genre["Id_Genre"] = genre.index + 1

rating = games[["Rating"]].drop_duplicates().reset_index(drop=True)
rating["Id_Rating"] = rating.index + 1

Avg_Time_Steam = games[['Id_Game', 'Average_Playtime']]


Avg_Time_Steam.columns = ['Id_Game', 'Average_Playtime']


Avg_Time_Steam = Avg_Time_Steam.dropna(subset=['Average_Playtime'])


Avg_Time_Steam = Avg_Time_Steam.reset_index(drop=True)

Region_Sales = games[['Id_Game', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']]

Region_Sales.columns = ['Id_Game', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']


# Create a new table with ID_Game and ID columns
games_main = games[['Id_Game', 'Name', 'Developer', 'Release', 'Platform', 'Genre', 'Rating', 'Score', 'Ranking',  'Global_Sales']]
games_main = games_main.merge(developer, on='Developer', how='left')
games_main = games_main.merge(platform, on='Platform', how='left')
games_main = games_main.merge(genre, on='Genre', how='left')
games_main = games_main.merge(rating, on='Rating', how='left')
columns_to_drop = ['Developer', 'Platform', 'Genre', 'Rating']
games_main = games_main.drop(columns_to_drop, axis=1)
column_order = ['Id_Game', 'Name', 'Release', 'Id_Developer', 'Id_Platform', 'Id_Genre', 'Id_Rating', 'Ranking', 'Score', 'Global_Sales']
games_main = games_main[column_order]

# games_main now contains the IDs and relationships

#Export tables to CSV files
developer.to_csv("developer.csv", index=False)
platform.to_csv("platform.csv", index=False)
genre.to_csv("genre.csv", index=False)
rating.to_csv("rating.csv", index=False)
games_main.to_csv("games_main.csv", index=False)
Region_Sales.to_csv("region_sales.csv", index=False)
Avg_Time_Steam.to_csv("avg_time_steam.csv", index=False)


In [1402]:
games_main.head()

,Id_Game,Name,Release,Id_Developer,Id_Platform,Id_Genre,Id_Rating,Ranking,Score,Global_Sales
0,1,Wii Sports,2006,1,1,1,1,46,8.0,82.53
1,2,Super Mario Bros.,1999,1,2,2,1,1087,NaN,5.07
2,2,Super Mario Bros.,1985,1,3,2,1,143,NaN,40.24
3,3,Mario Kart Wii,2008,1,1,3,1,170,8.3,35.52
4,4,Wii Sports Resort,2009,1,1,1,1,185,8.0,32.77
